In [ ]:
import json
import argparse

# 数据可视化控制参数
PLOT_FEATURES = True  # 设置为False可以跳过特征绘制，加快数据加载速度

# 命令行参数解析
parser = argparse.ArgumentParser()
parser.add_argument('--train_xj', type=str, default=None, help='训练轴承列表, 逗号分隔')
parser.add_argument('--test_xj', type=str, default=None, help='测试轴承列表, 逗号分隔')
parser.add_argument('--context_xj', type=str, default=None, help='上下文轴承列表, 逗号分隔')
parser.add_argument('--validation_xj', type=str, default=None, help='验证轴承列表, 逗号分隔')
parser.add_argument('--plot', action='store_true', help='是否绘制特征图')
args, unknown = parser.parse_known_args()

# 如果有命令行参数，则覆盖默认设置
if hasattr(args, 'plot') and args.plot:
    PLOT_FEATURES = args.plot

with open('../../config/fbtcn_config.json', 'r') as f:
    config = json.load(f)

print(f"🎨 特征可视化: {'开启' if PLOT_FEATURES else '关闭'}")

# 解析命令行参数，优先级高于config
if args.train_xj is not None:
    TRAIN_xj = [x.strip() for x in args.train_xj.split(',') if x.strip()]
else:
    TRAIN_xj = config['train_bearings']
if args.test_xj is not None:
    TEST_xj = [x.strip() for x in args.test_xj.split(',') if x.strip()]
else:
    TEST_xj = config['test_bearings']
if args.context_xj is not None:
    CONTEXT_xj = [x.strip() for x in args.context_xj.split(',') if x.strip()]
else:
    CONTEXT_xj = config['context_bearings']
if args.validation_xj is not None:
    VALIDATION_xj = [x.strip() for x in args.validation_xj.split(',') if x.strip()]
else:
    VALIDATION_xj = config['validation_bearings']

batch_size = config['batch_size']
test_batch_size = config['test_batch_size']
window_size = config['window_size']
input_dim = config['input_dim']
output_dim = config['output_dim']
num_channels = config['num_channels']
kernel_size = config['kernel_size']
dropout = config['dropout']
epochs = config['epochs']
learn_rate = config['learn_rate']
init_kl_weight = config['kl_weight']
forward_pass = config['forward_pass']
seed = config['seed']
patience = config['patience']
opt = config['opt']
output_posterior_rho_init = config['output_posterior_rho_init']
conv_posterior_rho_init = config['conv_posterior_rho_init']
if patience == "inf":
    patience = epochs
else:
    patience = int(patience)

In [ ]:
# 加载数据
import torch
from joblib import dump, load
import torch.utils.data as Data
import numpy as np
import pandas as pd
import torch.nn as nn
import os
import matplotlib.pyplot as plt

torch.autograd.set_detect_anomaly(True)
# 参数与配置
torch.manual_seed(seed)  # 设置随机种子，以使实验结果具有可重复性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"当前运行设备: {device}")

FPT_dict_xj = {'Bearing1_1': 76, 'Bearing1_2': 44, 'Bearing1_3': 60, 'Bearing1_4': 0, 'Bearing1_5': 39,
            'Bearing2_1': 455, 'Bearing2_2': 48, 'Bearing2_3': 327, 'Bearing2_4': 32, 'Bearing2_5': 141,
            'Bearing3_1': 2344, 'Bearing3_2': 0, 'Bearing3_3': 340, 'Bearing3_4': 1418, 'Bearing3_5': 9}

def plot_bearing_features(data, bearing_name, is_raw_data=True):
    """绘制轴承的11个特征"""
    feature_names = [
        'Kurtosis', 'Fractal Dimension', 'Peak Factor',
        'Energy Ratio', 'Spectral Flatness', 'Mean', 
        'Variance', 'Skewness', 'Peak Vibration', 'DE', 'FFT Mean'
    ]
    
    # 如果是tensor，转换为numpy
    if torch.is_tensor(data):
        data = data.numpy()
    
    # 确保数据形状正确
    if data.ndim == 3 and data.shape[1] == 1:  # (samples, 1, features)
        data = data.squeeze(1)  # 变为 (samples, features)
    
    plt.figure(figsize=(18, 12))
    plt.suptitle(f'{bearing_name} - 11个特征可视化', fontsize=16, y=0.98)
    
    for i in range(min(11, data.shape[1])):
        plt.subplot(4, 3, i+1)
        plt.plot(data[:, i], linewidth=1.2, alpha=0.8)
        plt.title(f'{feature_names[i]}', fontsize=12, pad=10)
        plt.xlabel('时间步')
        plt.ylabel('数值')
        plt.grid(True, alpha=0.3)
        
        # 添加统计信息
        mean_val = np.mean(data[:, i])
        std_val = np.std(data[:, i])
        plt.text(0.02, 0.98, f'μ={mean_val:.3f}\nσ={std_val:.3f}', 
                transform=plt.gca().transAxes, fontsize=8, 
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # 隐藏多余的子图
    for i in range(11, 12):
        plt.subplot(4, 3, i+1)
        plt.axis('off')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    plt.show()
    
    # 打印数据基本信息
    print(f"\n=== {bearing_name} 数据信息 ===")
    print(f"数据形状: {data.shape}")
    print(f"样本数量: {data.shape[0]}")
    print(f"特征数量: {data.shape[1]}")
    print(f"数据范围: [{np.min(data):.6f}, {np.max(data):.6f}]")
    print("-" * 50)

# 加载数据集
def dataloader(batch_size, test_batch_size, workers=os.cpu_count(), window_size=7, plot_features=True):
    """
    加载数据并可选择性绘制特征
    Args:
        plot_features: 是否绘制每个轴承的特征图
    """
    def get_data_and_labels(bearing_list, data_dir='../../datasetresult/xjtu_made', window_size=window_size, is_test=False, plot_features=True):
        data_list = []
        label_list = []
        
        for bearing in bearing_list:
            # 文件名格式: c*_Bearing*_*_data, c*_Bearing*_*_label
            data_files = [f for f in os.listdir(data_dir) if bearing in f and f.endswith('_data')]
            label_files = [f for f in os.listdir(data_dir) if bearing in f and f.endswith('_label')]
            
            # 按文件名排序，确保一一对应
            data_files.sort()
            label_files.sort()
            
            for data_file, label_file in zip(data_files, label_files):
                data = load(os.path.join(data_dir, data_file))
                label = load(os.path.join(data_dir, label_file))
                
                # 绘制特征（如果启用）
                if plot_features:
                    plot_bearing_features(data, bearing, is_raw_data=True)
                
                # 如果是测试集，且FPT_dict_xj中有该bearing，截取FPT值往后的数据
                # if is_test and bearing in FPT_dict_xj:
                #     fpt = FPT_dict_xj[bearing]
                #     if fpt > 0:
                #         data = data[fpt:]
                #         label = label[fpt:]
                
                data_list.append(data)
                label_list.append(label)
        
        # 拼接所有轴承的数据
        if len(data_list) > 0:
            data_all = torch.cat([torch.tensor(d, dtype=torch.float32) if not isinstance(d, torch.Tensor) else d for d in data_list], dim=0)
            label_all = torch.cat([torch.tensor(l, dtype=torch.float32) if not isinstance(l, torch.Tensor) else l for l in label_list], dim=0)
        else:
            data_all = torch.empty(0)
            label_all = torch.empty(0)
        
        return data_all, label_all

    print("=== 开始加载数据并绘制特征 ===")
    
    # 加载各个数据集
    print("\n📊 加载训练集数据...")
    train_set, train_label = get_data_and_labels(TRAIN_xj, plot_features=plot_features)
    # 获取并打印训练集第一个样本
    # train_set = train_set[0]
    # train_label = train_label[0]
    # print("\n📝 训练集第一个样本:")
    # print(f"形状: {train_set.shape}")
    # print(f"数据: {train_set}")


    print("\n📊 加载上下文数据...")
    context_set, context_label = get_data_and_labels(CONTEXT_xj, plot_features=plot_features)
    
    print("\n📊 加载测试集数据...")
    test_set, test_label = get_data_and_labels(TEST_xj, is_test=True, plot_features=plot_features)
    
    print("\n📊 加载验证集数据...")
    validation_set, validation_label = get_data_and_labels(VALIDATION_xj, plot_features=plot_features)
    
    print(f"\n=== 数据加载完成 ===")
    print(f"训练集形状: {train_set.shape}, {train_label.shape}")
    print(f"测试集形状: {test_set.shape}, {test_label.shape}")
    print(f"上下文集形状: {context_set.shape}, {context_label.shape}")
    print(f"验证集形状: {validation_set.shape}, {validation_label.shape}")
    
    # 创建数据加载器
    train_loader = Data.DataLoader(dataset=Data.TensorDataset(train_set, train_label),
                                   batch_size=batch_size, num_workers=workers, drop_last=False, shuffle=True)
    context_loader = Data.DataLoader(dataset=Data.TensorDataset(context_set, context_label),
                                   batch_size=batch_size, num_workers=workers, drop_last=False, shuffle=True)
    test_loader = Data.DataLoader(dataset=Data.TensorDataset(test_set, test_label),
                                  batch_size=test_batch_size, num_workers=workers, drop_last=False)
    validation_loader = Data.DataLoader(dataset=Data.TensorDataset(validation_set, validation_label),
                                   batch_size=test_batch_size, num_workers=workers, drop_last=False)
    
    return train_loader, context_loader, test_loader, validation_loader

# 加载数据并绘制特征
train_loader, context_loader, test_loader, validation_loader = dataloader(
    batch_size, test_batch_size, window_size=window_size, plot_features=PLOT_FEATURES
)

print(f"\n=== 数据加载器信息 ===")
print(f"训练集样本数: {len(train_loader.dataset)}")
print(f"上下文集样本数: {len(context_loader.dataset)}")
print(f"测试集样本数: {len(test_loader.dataset)}")
print(f"验证集样本数: {len(validation_loader.dataset)}")

In [3]:
# 定义裁剪模块
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size
    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

#  定义 TCN 卷积+残差 模块
from torch.nn.utils import parametrizations
import torch
import torch.nn as nn
from bayesian_torch.layers import Conv1dReparameterization, LinearReparameterization

import sys
sys.path.append('..')
from loss_function import compute_au_nll, compute_au_nll_with_pos
from function_kl import get_bayesian_model_parameters, get_bayesian_model_mu_rho, calculate_function_kl
from metrics import picp, nmpiw, ece, aleatoric_uncertainty, epistemic_uncertainty, ood_detection, sharpness,mae,rmse
from stable_fbtcn_training import model_train_stable, StabilizedAUNLL, get_stable_optimizer

class BayesianTemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding, dropout=0.2):
        super().__init__()
        self.conv1 = Conv1dReparameterization(
            in_channels, out_channels, kernel_size,
            stride=stride, padding=padding, dilation=dilation,
            prior_mean=0, prior_variance=1, posterior_mu_init=0, posterior_rho_init=conv_posterior_rho_init
        )
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.conv2 = Conv1dReparameterization(
            out_channels, out_channels, kernel_size,
            stride=stride, padding=padding, dilation=dilation,
            prior_mean=0, prior_variance=1, posterior_mu_init=0, posterior_rho_init=conv_posterior_rho_init
        )
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        # self.downsample = Conv1dReparameterization(
        #     in_channels, out_channels, 1,
        #     prior_mean=0, prior_variance=1, posterior_mu_init=0, posterior_rho_init=-3
        # ) if in_channels != out_channels else None
        self.final_relu = nn.ReLU()

    def forward(self, x):
        kl = 0.0
        out, kl1 = self.conv1(x)
        out = self.chomp1(out)
        kl = kl + kl1
        out = self.relu1(out)
        out = self.dropout1(out)
        out, kl2 = self.conv2(out)
        out = self.chomp2(out)
        kl = kl + kl2
        out = self.relu2(out)
        out = self.dropout2(out)
        res = x if self.downsample is None else self.downsample(x)
        return self.final_relu(out + res), kl
    


In [4]:
# 定义一个真正的高斯过程输出层
class GaussianProcessOutputLayer(nn.Module):
    """
    真正的高斯过程回归输出层
    使用RBF核函数进行高斯过程回归
    """
    def __init__(self, in_features, out_features, num_inducing_points=50, kernel_lengthscale=1.0, kernel_variance=1.0, noise_variance=0.1):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_inducing_points = num_inducing_points
        
        # 诱导点 (inducing points) - 这些是GP的"支持点"
        self.inducing_points = nn.Parameter(torch.randn(num_inducing_points, in_features))
        
        # 诱导点对应的函数值
        self.inducing_values = nn.Parameter(torch.randn(num_inducing_points, out_features))
        
        # 核函数参数
        self.log_lengthscale = nn.Parameter(torch.log(torch.tensor(kernel_lengthscale)))
        self.log_kernel_variance = nn.Parameter(torch.log(torch.tensor(kernel_variance)))
        self.log_noise_variance = nn.Parameter(torch.log(torch.tensor(noise_variance)))
        
        # 用于数值稳定性的jitter
        self.jitter = 1e-6
        
    def rbf_kernel(self, x1, x2):
        """
        RBF (Radial Basis Function) 核函数
        k(x1, x2) = σ² * exp(-0.5 * ||x1 - x2||² / l²)
        """
        lengthscale = torch.exp(self.log_lengthscale)
        kernel_variance = torch.exp(self.log_kernel_variance)
        
        # 计算距离矩阵
        # x1: (n1, d), x2: (n2, d)
        dist_sq = torch.cdist(x1, x2, p=2).pow(2)
        
        # RBF核
        kernel_matrix = kernel_variance * torch.exp(-0.5 * dist_sq / (lengthscale ** 2))
        return kernel_matrix
        
    def forward(self, x):
        """
        高斯过程前向传播
        x: (batch_size, in_features)
        返回: (mu, sigma) 其中sigma是标准差而不是方差
        """
        batch_size = x.shape[0]
        noise_variance = torch.exp(self.log_noise_variance)
        
        # 计算核矩阵
        K_uu = self.rbf_kernel(self.inducing_points, self.inducing_points)  # (m, m)
        K_uf = self.rbf_kernel(self.inducing_points, x)  # (m, n)
        K_ff_diag = torch.exp(self.log_kernel_variance).expand(batch_size)  # (n,)对角元素
        
        # 添加噪声和jitter以确保数值稳定性
        K_uu_inv = torch.inverse(K_uu + (noise_variance + self.jitter) * torch.eye(self.num_inducing_points, device=x.device))
        
        # GP预测均值: μ* = K_uf^T K_uu^(-1) f_u
        mu = torch.matmul(K_uf.t(), torch.matmul(K_uu_inv, self.inducing_values))  # (n, out_features)
        
        # GP预测方差: σ²* = K_ff - K_uf^T K_uu^(-1) K_uf + noise
        # 这里我们计算对角元素（每个预测点的方差）
        K_uf_Kuu_inv = torch.matmul(K_uu_inv, K_uf)  # (m, n)
        var_reduction = torch.sum(K_uf * K_uf_Kuu_inv, dim=0)  # (n,)
        variance = K_ff_diag - var_reduction + noise_variance  # (n,)
        
        # 确保方差为正
        variance = torch.clamp(variance, min=self.jitter)
        sigma = torch.sqrt(variance).unsqueeze(-1).expand(-1, self.out_features)  # (n, out_features)
        
        return mu, sigma


In [5]:
import torch.nn.functional as F

attention_dim = 1  # 因为输入的最后一维是1
class SelfAttention(nn.Module):
    def __init__(self, embed_dim):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        # 将输入维度从1映射到attention_dim
        self.query = nn.Linear(1, embed_dim)
        self.key = nn.Linear(1, embed_dim) 
        self.value = nn.Linear(1, embed_dim)
        
    def forward(self, x):
        # x shape: (batch_size=1024, seq_len=128, features=1)
        # 对最后一维进行线性变换
        q = self.query(x)  # (1024, 128, embed_dim)
        k = self.key(x)    # (1024, 128, embed_dim)
        v = self.value(x)  # (1024, 128, embed_dim)
        
        # 计算注意力权重
        attention_weights = F.softmax(torch.matmul(q, k.transpose(1, 2)) / torch.sqrt(torch.tensor(self.embed_dim)), dim=-1)
        # 应用注意力权重
        output = torch.matmul(attention_weights, v)  # (1024, 128, embed_dim)
        return output

In [6]:
class BayesianTCN(nn.Module):
    def __init__(self, input_dim, num_channels, attention_dim, kernel_size=2, dropout=0.2, output_dim=1):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        self.kl_modules = []
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = input_dim if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            block = BayesianTemporalBlock(
                in_channels, out_channels, kernel_size,
                stride=1, dilation=dilation_size,
                padding=(kernel_size-1)*dilation_size,
                dropout=dropout
            )
            layers.append(block)
        self.network = nn.ModuleList(layers)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.mu = LinearReparameterization(
            in_features=num_channels[-1], out_features=output_dim,
            prior_mean=0, prior_variance=1, posterior_mu_init=0, posterior_rho_init=output_posterior_rho_init  # 增大初始后验方差，使均值预测更具波动性
        )
        # posterior_rho_init=-5时，后验的对数方差为-10 因为对数方差 log(var) = 2 * rho，rho=-5，所以 log(var)=2*(-5)=-10
        self.sigma = LinearReparameterization(
            in_features=num_channels[-1], out_features=output_dim,
            prior_mean=0, prior_variance=1, posterior_mu_init=0, posterior_rho_init=output_posterior_rho_init
        )

        # 定义自注意力层
        self.attention = SelfAttention(attention_dim)

        # 自适应平均池化
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)

    def generate_init_params(self, sample_input):
        """
        根据输入数据的形状初始化模型参数
        Args:
            sample_input: 输入数据的一个样本（用于推断参数形状）
        Returns:
            params_dict: 包含所有参数的字典
        """
        # 确保不计算梯度
        with torch.no_grad():
            # 前向传播一次（不保存计算图）
            _ = self.forward(sample_input)
            params_dict = {k: v.clone() for k, v in self.state_dict().items()}
            return params_dict
        
    def forward(self, x, feature=False):
        # x: [batch, seq_len, input_dim] -> [batch, input_dim, seq_len]
        x = x.permute(0, 2, 1)
        kl = 0.0
        for block in self.network:
            x, kl_block = block(x)
            kl = kl + kl_block
        # x = self.avgpool(x)
        x = self.attention(x)
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)

        mu, kl_mu = self.mu(x)
        sigma, kl_sigma = self.sigma(x) # sigma是对数方差
        # sigma = torch.log(sigma ** 2) + 1e-8 # 显式地将方差取对数
        sigma = F.softplus(sigma) # sigma是对数方差, softplus保证sigma>0
        kl = kl + (kl_mu+kl_sigma)
        if feature:
            return mu, sigma, kl, x
        else:
            return mu, sigma, kl

    def kl_loss(self):
        kl = 0.0
        for m in self.children():  # 只遍历直接子模块
            if hasattr(m, "kl_loss"):
                kl = kl + m.kl_loss()
        return kl

In [ ]:
import copy

model = BayesianTCN(
    input_dim=input_dim,        
    num_channels=num_channels,  # 这里必须是列表
    attention_dim=attention_dim,
    kernel_size=kernel_size,
    dropout=dropout,
    output_dim=output_dim
)
init_model = copy.deepcopy(model)
print(model)
# 定义损失函数和优化函数 
# loss_function = nn.MSELoss()  # loss
loss_function = compute_au_nll_with_pos

# 看下这个网络结构总共有多少个参数
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')

count_parameters(model)

# 训练过程

In [8]:
# # 训练模型 - 包含验证集
# import time
# import torch.nn.functional as F
# import matplotlib.pyplot as plt
# import itertools

# def model_train_with_validation(epochs, model, optimizer, loss_function, train_loader, context_loader, validation_loader, device):
#     model = model.to(device)
#     # 最低验证损失
#     minimum_train_loss = float('inf')
#     # 最佳模型
#     best_model_state = None
#     train_losses = []     # 记录在训练集上每个epoch的损失
#     val_losses = []       # 记录在验证集上每个epoch的损失
#     # 计算模型运行时间
#     start_time = time.time()
#     patience_counter = 0
#     for epoch in range(epochs):
#         # ===================== 训练阶段 =====================
#         model.train()
#         train_loss = []    # 保存当前epoch的训练损失
#         kl_losses = []     # 保存当前epoch的KL损失
#         nll_losses = []    # 保存当前epoch的NLL损失
#         # 创建循环的context_loader迭代器
#         context_iter = itertools.cycle(context_loader)
        
#         # 遍历train_loader，并从循环的context_iter中获取对应的context数据
#         for i, train_batch in enumerate(train_loader):
#             seq, labels = train_batch
#             context_batch = next(context_iter)
#             context_seq, _ = context_batch
#             seq, labels = seq.to(device), labels.to(device)
#             context_seq = context_seq.to(device)
            
#             # 每次更新参数前都梯度归零
#             optimizer.zero_grad()
            
#             # 前向传播
#             mu, sigma, _ = model(seq)
#             params_variational_mean, params_variational_logvar = get_bayesian_model_mu_rho(model)
#             kl_loss = calculate_function_kl(params_variational_mean, params_variational_logvar, context_seq, model=model)
            
#             # 损失计算
#             nll_loss = loss_function(labels, mu, sigma)
#             kl_weight = max(init_kl_weight, 1.0 * (1 - epoch/epochs))
#             loss = nll_loss + kl_weight * kl_loss
            
#             # 检查NaN或Inf
#             if torch.isnan(loss) or torch.isinf(loss):
#                 print(f"Warning: Invalid loss detected at epoch {epoch+1}, batch {i+1}")
#                 continue
                
#             train_loss.append(loss.item())
#             kl_losses.append(kl_loss.item())
#             nll_losses.append(nll_loss.item())
#             # 反向传播和参数更新
#             loss.backward()
#             optimizer.step()
#         # 更新学习率
#         scheduler.step()
#         # ===================== 记录和显示损失 =====================
#         if len(train_loss) > 0:
#             train_avg_loss = np.average(train_loss)
#             train_avg_kl = np.average(kl_losses) if len(kl_losses) > 0 else 0.0
#             train_avg_nll = np.average(nll_losses) if len(nll_losses) > 0 else 0.0
#             train_losses.append(train_avg_loss)
#             # 训练集sigma
#             if 'sigma' in locals():
#                 train_sigma_mean = sigma.mean().item()
#             else:
#                 train_sigma_mean = float('nan')
#             print(f'Epoch: {epoch+1:3} | Train Loss: {train_avg_loss:8.6f} | Train KL: {train_avg_kl:8.2f} | Train NLL: {train_avg_nll:8.2f} | Train Sigma Mean: {train_sigma_mean:.6f}')
#         else:
#             print(f'Epoch: {epoch+1:3} - No valid batches processed')
#             continue
       
#         # ===================== 早停和最佳模型保存 =====================
#         # 使用验证损失进行早停判断
#         if train_avg_loss < minimum_train_loss:
#             minimum_train_loss = train_avg_loss
#             best_model_state = model.state_dict().copy()  # 深拷贝模型状态
#             best_epoch = epoch + 1
#             patience_counter = 0
#             print(f"✓ New best model at epoch {epoch+1} with train loss: {train_avg_loss:.6f}")
#         else:
#             patience_counter += 1
#         if patience_counter >= patience:
#             print(f"Early stopping at epoch {epoch+1}, best epoch was {best_epoch} with train loss: {minimum_train_loss:.6f}")
#             break
    
#     # 保存最好的模型参数
#     if best_model_state is not None:
#         torch.save(best_model_state, 'best_model_fbtcn.pt')
#         print(f"Best model saved with train loss: {minimum_train_loss:.6f}")
    
#     print(f'\nDuration: {time.time() - start_time:.0f} seconds')
    
#     # ===================== 可视化训练过程 =====================
#     plt.figure(figsize=(12, 5))
    
#     # 子图1: 损失曲线
#     plt.subplot(1, 2, 1)
#     epochs_range = range(1, len(train_losses) + 1)
#     plt.plot(epochs_range, train_losses, 'b-', label='Train Loss', linewidth=2)
#     # plt.plot(epochs_range, val_losses, 'r-', label='Validation Loss', linewidth=2)
#     plt.axvline(x=best_epoch, color='g', linestyle='--', alpha=0.7, label=f'Best Epoch ({best_epoch})')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.title('Training Loss')
#     plt.legend()
#     plt.grid(True, alpha=0.3)
    
#     # 子图2: 放大后期损失变化
#     if len(train_losses) > 20:
#         plt.subplot(1, 2, 2)
#         start_idx = len(train_losses) // 2  # 从中间开始显示
#         plt.plot(epochs_range[start_idx:], train_losses[start_idx:], 'b-', label='Train Loss', linewidth=2)
#         # plt.plot(epochs_range[start_idx:], val_losses[start_idx:], 'r-', label='Validation Loss', linewidth=2)
#         if best_epoch > start_idx:
#             plt.axvline(x=best_epoch, color='g', linestyle='--', alpha=0.7, label=f'Best Epoch ({best_epoch})')
#         plt.xlabel('Epoch')
#         plt.ylabel('Loss')
#         plt.title('Loss (Later Half)')
#         plt.legend()
#         plt.grid(True, alpha=0.3)
    
#     plt.tight_layout()
#     plt.show()
    
#     print(f'Minimum train loss: {minimum_train_loss:.6f}')
#     return train_losses, val_losses, best_epoch

# #  模型训练 - 现在包含验证集
# print("开始训练模型（包含验证集）...")
# train_losses, val_losses, best_epoch = model_train_with_validation(
#     epochs, model, optimizer, loss_function, 
#     train_loader, context_loader, validation_loader, device
# )

# 稳定训练过程

In [ ]:

optimizer = get_stable_optimizer(model, config)
# 替换原始训练函数 - 添加跳过验证集的选项
train_losses, val_losses, best_epoch = model_train_stable(
    epochs, model, init_model, optimizer,  compute_au_nll,
    train_loader, context_loader, validation_loader, device, config,
    skip_validation=True,  # 设置为True跳过验证集计算，加快训练速度
)

# 测试集评估

In [ ]:
# %% [markdown]
# 模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载模型的状态字典
model.load_state_dict(torch.load('best_model_fbtcn_stable.pt'))
model = model.to(device)

# 预测数据
test_origin_data = []
test_pre_data = []
mu_std_list = []  # 修复：直接存储每个样本的std值，而不是每个batch的列表

with torch.no_grad():
    for data, label in test_loader:
        # 原始标签
        origin_lable = label.tolist()
        test_origin_data += origin_lable
        model.eval()  # 将模型设置为评估模式 
        # 执行N次前向传播，并计算metrics中的指标
        data, label = data.to(device), label.to(device)
        mu_list = []
        sigma_list = []

        with torch.no_grad():
            for _ in range(forward_pass):
                mu, sigma, kl = model(data)
                mu_list.append(mu.cpu().numpy())
                sigma_list.append(sigma.cpu().numpy())
        mu_samples = np.stack(mu_list, axis=0)  # (N, batch, 1)
        sigma_samples = np.stack(sigma_list, axis=0)  # (N, batch, 1)
        # 计算均值和方差
        mu_mean = np.mean(mu_samples, axis=0)  # (batch, 1)
        mu_std = np.std(mu_samples, axis=0)    # (batch, 1)
        # 记录预测均值
        test_pre_data += mu_mean.squeeze(-1).tolist()
        # 修复：使用extend而不是append，避免嵌套列表造成的形状不一致问题
        mu_std_list.extend(mu_std.squeeze(-1).tolist())
        # 记录原始标签
        # test_origin_data += label.cpu().numpy().tolist()  # 已在外层添加
        # 计算metrics
        # 只在最后一批时打印一次指标
        # if len(test_pre_data) >= len(test_loader.dataset):
print(len(test_origin_data))
print(len(test_pre_data))
print(f"mu_std_list length: {len(mu_std_list)}")  # 验证长度是否正确

In [ ]:
import os
scaler_dir = '../../datasetresult/xjtu_made'
scaler_filename = f'{TEST_xj[0]}_labeled_scaler'
all_files = os.listdir(scaler_dir)
matched_files = [f for f in all_files if f.endswith(scaler_filename)]
if not matched_files:
    raise FileNotFoundError(f"未找到匹配 {TEST_xj[0]} 的 scaler 文件, 目录: {scaler_dir}")
scaler_path = os.path.join(scaler_dir, matched_files[0])
scaler = load(scaler_path)

# test_origin_data = scaler.inverse_transform(test_origin_data)
# test_pre_data = scaler.inverse_transform(test_pre_data)

print(matched_files)
print(len(test_origin_data))
print(len(test_pre_data))
test_origin_data = scaler.inverse_transform(np.array(test_origin_data).reshape(-1, 1)).reshape(-1)
test_pre_data = scaler.inverse_transform(np.array(test_pre_data).reshape(-1, 1)).reshape(-1)


import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 模型分数
score = r2_score(test_origin_data, test_pre_data)
print('测试集上 模型分数-R^2:',score)

print('*'*50)
# 训练集上的预测误差
test_mse = mean_squared_error(test_origin_data, test_pre_data)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(test_origin_data, test_pre_data)

y_true = np.array(test_origin_data).reshape(-1)
y_pred_mean = np.array(test_pre_data).reshape(-1)
# 获取test_pre_data的上下界（置信区间）
from scipy.stats import norm
z = norm.ppf(1 - 0.05 / 2)  # 95%置信区间
# 由于test_pre_data已反归一化，mu_std_list需乘以scaler.scale_得到原空间std
if hasattr(scaler, 'scale_'):
    std_scale = scaler.scale_[0] if scaler.scale_.ndim > 0 else scaler.scale_
else:
    std_scale = 1.0

# 修复：现在mu_std_list已经是一维列表，直接转换为numpy数组即可
y_pred_std_origin = np.array(mu_std_list) * std_scale
y_pred_mean_origin = np.array(test_pre_data).reshape(-1)
test_pre_data_lower = y_pred_mean_origin - z * y_pred_std_origin
test_pre_data_upper = y_pred_mean_origin + z * y_pred_std_origin

y_pred_std = np.array(mu_std_list)  # 修复：直接转换，无需reshape  

print(f"{'指标名称':<25} | {'数值'}")
print("-" * 40)
print(f"{'均方误差-MSE':<25} | {test_mse:.6f}")
print(f"{'均方根误差-RMSE':<25} | {test_rmse:.6f}")
print(f"{'平均绝对误差-MAE':<25} | {test_mae:.6f}")
print(f"{'PICP':<25} | {picp(y_true, test_pre_data_lower, test_pre_data_upper):.6f}")
print(f"{'NMPIW':<25} | {nmpiw(y_true, y_pred_mean, y_pred_std):.6f}")
print(f"{'ECE':<25} | {ece(y_true, y_pred_mean, y_pred_std):.6f}")
print(f"{'Aleatoric Uncertainty':<25} | {aleatoric_uncertainty(y_pred_std):.6f}")
print(f"{'Epistemic Uncertainty':<25} | {epistemic_uncertainty(mu_samples.squeeze(-1)):.6f}")
print(f"{'Sharpness':<25} | {sharpness(y_pred_std):.6f}")

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# 可视化结果
plt.figure(figsize=(12, 6), dpi=100)
plt.plot(test_origin_data, label='Label',color='pink')  # 真实值
plt.plot(test_pre_data, label='Prediction',color='c')  # 预测值
# 计算预测区间（假设你有y_pred_mean和y_pred_std，且已反归一化）
# 这里我们假设 test_pre_data 是反归一化后的 y_pred_mean
# 如果有 y_pred_std（标准差），也需要反归一化
# 若 mu_std 是标准化空间下的 std，需乘以 scaler.scale_ 得到原空间 std

# 检查是否有 mu_std_list，如果有则反归一化
if 'mu_std_list' in locals():
    # mu_std: 标准化空间下的 std
    # scaler.scale_ 是 shape (1,) 或 (n_features,)
    if hasattr(scaler, 'scale_'):
        std_scale = scaler.scale_[0] if scaler.scale_.ndim > 0 else scaler.scale_
    else:
        std_scale = 1.0
    y_pred_std_origin = np.array(mu_std_list) * std_scale  
    y_pred_mean_origin = test_pre_data  # 已反归一化
    # 计算95%置信区间
    from scipy.stats import norm
    z = norm.ppf(1 - 0.05 / 2)
    lower = y_pred_mean_origin - z * y_pred_std_origin
    upper = y_pred_mean_origin + z * y_pred_std_origin
    # 绘制置信区间
    plt.fill_between(np.arange(len(y_pred_mean_origin)), lower, upper, color='c', alpha=0.2, label='95% CI')

# plt.plot([-1,170],[2.0*0.7,2.0*0.7],c='black',lw=1,ls='--')  # 临界点直线,可自己调整位置

plt.xlabel('Cycle/10s', fontsize=12)
plt.ylabel('RUL', fontsize=12)
plt.title(TEST_xj[0] +' Prediction', fontsize=16)
plt.legend()
plt.savefig(f'../画图对比/{TEST_xj[0]}_fbtcn.png')

# 保存数据
dump(test_origin_data, '../画图对比/fbtcn_origin') 
dump(test_pre_data, '../画图对比/fbtcn_pre') 